In [1]:
# Load in packages
library(tidyverse)
library(data.table)
library(readxl)
library(dplyr)
library(stringr)
library(ggplot2)
install.packages("ggbreak")
library(ggbreak)
library(tidyr)
install.packages("ggpubr")
library(ggpubr)

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The follow

# Clear environment: 

In [ ]:
rm(list=ls())
setwd("")

# Set your datasets below and read in the files containing all indels ≤ 50bp for each dataset:

In [ ]:
db_name_1 = "gnomAD"
db_name_2 = "IGM"
db_name_3 = "UK.BB"

df_1 = fread("gnomad.exomes.r2.1.1.sites_indelsonly_rAF_lt50bp.csv")
df_2 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_lt50bp.csv")
df_3 = fread("UK.BB.exomes.430k.sites_indelsonly_rAF_lt50bp.csv")

# Read in rAF_hi indels file for each dataset: 

In [ ]:
rAF_hi_indels_10bp_df_1 = fread("gnomad.exomes.r2.1.1.sites_indelsonly_rAF_bp10_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_20bp_df_1 = fread("gnomad.exomes.r2.1.1.sites_indelsonly_rAF_bp20_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_30bp_df_1 = fread("gnomad.exomes.r2.1.1.sites_indelsonly_rAF_bp30_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_40bp_df_1 = fread("gnomad.exomes.r2.1.1.sites_indelsonly_rAF_bp40_rAF_hiIndels.lt50bp.csv")

rAF_hi_indels_10bp_df_2 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp10_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_20bp_df_2 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp20_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_30bp_df_2 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp30_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_40bp_df_2 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp40_rAF_hiIndels.lt50bp.csv")

rAF_hi_indels_10bp_df_3 = fread("UK.BB.exomes.430k.sites_indelsonly_rAF_bp10_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_20bp_df_3 = fread("UK.BB.exomes.430k.sites_indelsonly_rAF_bp20_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_30bp_df_3 = fread("UK.BB.exomes.430k.sites_indelsonly_rAF_bp30_rAF_hiIndels.lt50bp.csv")
rAF_hi_indels_40bp_df_3 = fread("UK.BB.exomes.430k.sites_indelsonly_rAF_bp40_rAF_hiIndels.lt50bp.csv")


# Declare your bp range:


In [ ]:
bp_range = c("10", "20", "30", "40")

# Separate your VarID into CHR, POS, REF, ALT



In [ ]:
df_1 = separate(df_1, col = VarID, into = c("CHR", "POS", "REF", "ALT"), sep = "-")
df_2 = separate(df_2, col = VarID, into = c("CHR", "POS", "REF", "ALT"), sep = "-")
df_3 = separate(df_3, col = VarID, into = c("CHR", "POS", "REF", "ALT"), sep = "-")

# Calculate region length and create individual graphs: 

In [ ]:
for (i in bp_range) {
  print(i)
  
  ## Find the regions that the rAF_hi indels are in
  temp_regions_1 = distinct(get(paste0("rAF_hi_indels_", i, "bp_df_1")) %>% select(paste0("GID_bp", i)))
  temp_regions_2 = distinct(get(paste0("rAF_hi_indels_", i, "bp_df_2")) %>% select(paste0("GID_bp", i)))
  temp_regions_3 = distinct(get(paste0("rAF_hi_indels_", i, "bp_df_3")) %>% select(paste0("GID_bp", i)))
  
  temp_var = paste0("GID_bp", i)
  
  temp_df_1 = paste0("filtered_df_1_", i)
  temp_df_2 = paste0("filtered_df_2_", i)
  temp_df_3 = paste0("filtered_df_3_", i)
  
  ## Keep the regions that have at least one rAF_hi indel and have to have at least 2 indels (the second indel doesn't have to be rAF_hi)
  assign(temp_df_1,  df_1 %>% filter(df_1[[temp_var]] %in% temp_regions_1[[temp_var]]) %>% group_by(get(temp_var)) %>% filter(n()>1))
  assign(temp_df_2,  df_2 %>% filter(df_2[[temp_var]] %in% temp_regions_2[[temp_var]]) %>% group_by(get(temp_var)) %>% filter(n()>1))
  assign(temp_df_3,  df_3 %>% filter(df_3[[temp_var]] %in% temp_regions_3[[temp_var]]) %>% group_by(get(temp_var)) %>% filter(n()>1))
  
  #find start and end indels 
  assign(temp_df_1, get(temp_df_1) %>% group_by(get(temp_var)) %>% mutate(start_indel = as.numeric(min(POS)), end_indel = as.numeric(max(POS))) %>% ungroup())
  assign(temp_df_2, get(temp_df_2) %>% group_by(get(temp_var)) %>% mutate(start_indel = as.numeric(min(POS)), end_indel = as.numeric(max(POS))) %>% ungroup())
  assign(temp_df_3, get(temp_df_3) %>% group_by(get(temp_var)) %>% mutate(start_indel = as.numeric(min(POS)), end_indel = as.numeric(max(POS))) %>% ungroup())
  
  ## calculate the length of region 
  assign(temp_df_1, get(temp_df_1) %>% mutate (region_length = get(temp_df_1)$end_indel - get(temp_df_1)$start_indel))
  assign(temp_df_2, get(temp_df_2) %>% mutate (region_length = get(temp_df_2)$end_indel - get(temp_df_2)$start_indel))
  assign(temp_df_3, get(temp_df_3) %>% mutate (region_length = get(temp_df_3)$end_indel - get(temp_df_3)$start_indel))
  
  ## Get a chart with the number of regions with that region length
  temp_chart_1 = paste0("region_len_1_", i)
  temp_chart_2 = paste0("region_len_2_", i)
  temp_chart_3 = paste0("region_len_3_", i)
  
  assign(temp_chart_1, distinct(get(temp_df_1) %>% select (paste0("GID_bp", i), "region_length")))
  assign(temp_chart_2, distinct(get(temp_df_2) %>% select (paste0("GID_bp", i), "region_length")))
  assign(temp_chart_3, distinct(get(temp_df_3) %>% select (paste0("GID_bp", i), "region_length")))
  
  ## Get a graph 
  graph_df1 = paste0("graph_df1_bp", i)
  graph_df2 = paste0("graph_df2_bp", i)
  graph_df3 = paste0("graph_df3_bp", i)


  ## Make graphs 
  assign(graph_df1, ggplot(get(temp_chart_1), aes(x = region_length)) +
           geom_histogram(aes(y = ..density..), colour= "black", fill = "white", regionwidth = 1) +
           geom_density(fill="grey", alpha = .5) +
           scale_x_continuous(breaks =seq(from = 0, to = 1000, by = 20), limits = c(0, 1000)) +
           scale_x_break(breaks = c(80, 800), scales = 0.25, ticklabels = c(800, 900, 1000)) +
           ylim(0, 0.3) +
           labs(title = paste0(db_name_1, ": ", i, "bp region"), x = "region Lengths (bps)") +
           theme(axis.text=element_text(size=20), axis.text.x = element_text(angle = 90), axis.title=element_text(size=20), plot.title=element_text(size=32, hjust = 0.5, margin = margin( 1, 1, 1, 1, "cm")), plot.margin = margin( 2, 2, 2, 2, "cm"), axis.title.y = element_text(margin = margin( 1, 1, 1, 1, "cm"), angle = 90)))

  assign(graph_df2, ggplot(get(temp_chart_2), aes(x = region_length)) +
           geom_histogram(aes(y = ..density..), colour= "black", fill = "white", regionwidth = 1) +
           geom_density(fill="grey", alpha = .5) +
           scale_x_continuous(breaks =seq(from = 0, to = 1000, by = 20), limits = c(0, 1000)) +
           scale_x_break(breaks = c(80, 800), scales = 0.25, ticklabels = c(800, 900, 1000)) +
           ylim(0, 0.3) +
           labs(title = paste0(db_name_2, ": ", i, "bp region"), x = "region Lengths (bps)") +
           theme(axis.text=element_text(size=20),  axis.text.x = element_text(angle = 90), axis.title=element_text(size=20), plot.title=element_text(size=32, hjust = 0.5, margin = margin( 1, 1, 1, 1, "cm")), plot.margin = margin( 2, 2, 2, 2, "cm"), axis.title.y = element_text(margin = margin( 1, 1, 1, 1, "cm"), angle = 90)))

  assign(graph_df3, ggplot(get(temp_chart_3), aes(x = region_length)) +
           geom_histogram(aes(y = ..density..), colour= "black", fill = "white", regionwidth = 1) +
           geom_density(fill="grey", alpha = .5) + 
           scale_x_continuous(breaks =seq(from = 0, to = 1000, by = 20), limits = c(0, 1000)) + 
           scale_x_break(breaks = c(80, 800), scales = 0.25, ticklabels = c(800, 900, 1000)) +
           ylim(0, 0.3) + 
           labs(title = paste0(db_name_3, ": ", i, "bp region"), x = "region Lengths (bps)") + 
           theme(axis.text=element_text(size=20),  axis.text.x = element_text(angle = 90), axis.title=element_text(size=20), plot.title=element_text(size=32, hjust = 0.5, margin = margin( 1, 1, 1, 1, "cm")), plot.margin = margin( 2, 2, 2, 2, "cm"), axis.title.y = element_text(margin = margin( 1, 1, 1, 1, "cm"), angle = 90)))
}

In [ ]:

ggarrange(print(graph_df1_bp10),
          print(graph_df1_bp20),
          print(graph_df1_bp30),
          print(graph_df1_bp40),
          print(graph_df2_bp10),
          print(graph_df2_bp20),
          print(graph_df2_bp30),
          print(graph_df2_bp40),
          print(graph_df3_bp10),
          print(graph_df3_bp20),
          print(graph_df3_bp30),
          print(graph_df3_bp40),
          labels = c("A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"),
          ncol = 4, nrow = 2, font.label = list(size = 32, color = "black")) + theme(plot.margin = unit(c(2,2,2,2), "cm"))
ggsave("Fig2.jpg", width = 80, height = 40, units = c("cm"), dpi = 300)



# Written Results Section


In [ ]:
## To find the mean and median region lengths. 
for (i in bp_range) {
  # get the dataframe with all the indels in a region with a rAF_hi indel 
  temp_var1 = paste0("filtered_df_1_", i)
  temp_var2 = paste0("filtered_df_2_", i)
  temp_var3 = paste0("filtered_df_3_", i)
  
  # find mean and median 
  mean_df1 = paste0("mean_region_len_df1_bp", i)
  mean_df2 = paste0("mean_region_len_df2_bp", i)
  mean_df3 = paste0("mean_region_len_df3_bp", i)
  
  gID = paste0("GID_bp", i)
  
  
  ## remember: got to collapse by gID 
  assign(mean_df1, mean(unique(get(temp_var1) %>% select(gID, region_length))$region_length))
  assign(mean_df2, mean(unique(get(temp_var2) %>% select(gID, region_length))$region_length))
  assign(mean_df3, mean(unique(get(temp_var3) %>% select(gID, region_length))$region_length))
  
  median_df1 = paste0("median_region_len_df1_bp", i)
  median_df2 = paste0("median_region_len_df2_bp", i)
  median_df3 = paste0("median_region_len_df3_bp", i)
  
  assign(median_df1, median(unique(get(temp_var1) %>% select(gID, region_length))$region_length))
  assign(median_df2, median(unique(get(temp_var2) %>% select(gID, region_length))$region_length))
  assign(median_df3, median(unique(get(temp_var3) %>% select(gID, region_length))$region_length))
  
  print(paste0("Mean region length for ", db_name_1, " and bp region ", i, ": ", get(mean_df1)))
  print(paste0("Mean region length for ", db_name_2, " and bp region ", i, ": ", get(mean_df2)))
  print(paste0("Mean region length for ", db_name_3, " and bp region ", i, ": ", get(mean_df3)))
  
  print(paste0("Median region length for ", db_name_1, " and bp region ", i, ": ", get(median_df1)))
  print(paste0("Median region length for ", db_name_2, " and bp region ", i, ": ", get(median_df2)))
  print(paste0("Median region length for ", db_name_3, " and bp region ", i, ": ", get(median_df3)))
}


## To find the percentage of regions with rAF_hi indels with regions < X bp for each bp region

In [ ]:

for (i in bp_range) {
  
  ## get the region_len dataframe 
  ben_len_1 = paste0("region_len_1_", i)
  ben_len_2 = paste0("region_len_2_", i)
  ben_len_3 = paste0("region_len_3_", i)
  
  ## assign the count region variable 
  count_region_1 = paste0("count_region", i, "_df1")
  count_region_2 = paste0("count_region", i, "_df2")
  count_region_3 = paste0("count_region", i, "_df3")
  
  assign(count_region_1, get(ben_len_1) %>% group_by(region_length) %>% count(region_length))
  assign(count_region_2, get(ben_len_2) %>% group_by(region_length) %>% count(region_length))
  assign(count_region_3, get(ben_len_3) %>% group_by(region_length) %>% count(region_length))
  
  ## find the percentage 
  percent_df1 = paste0("percent_lt_", i, "bp_region_", i, "df1")
  percent_df2 = paste0("percent_lt_", i, "bp_region_", i, "df2")
  percent_df3 = paste0("percent_lt_", i, "bp_region_", i, "df3")
  
  assign(percent_df1, sum((get(count_region_1) %>% filter(region_length < as.numeric(i)))$n) / sum(get(count_region_1)$n) * 100 )
  assign(percent_df2, sum((get(count_region_2) %>% filter(region_length < as.numeric(i)))$n) / sum(get(count_region_2)$n) * 100 )
  assign(percent_df3, sum((get(count_region_3) %>% filter(region_length < as.numeric(i)))$n) / sum(get(count_region_3)$n) * 100 )

  print(paste0("Percent less than ", i, " bps, region ", i, " is ", get(percent_df1), "% for dataset ", db_name_1))
  print(paste0("Percent less than ", i, " bps, region ", i, " is ", get(percent_df2), "% for dataset ", db_name_2))
  print(paste0("Percent less than ", i, " bps, region ", i, " is ", get(percent_df3), "% for dataset ", db_name_3))
  
}


## We observed XX regions with rAF_hi indels using the 10bp sliding region, and XX, XX, XX for the 20bp, 30bp and 40bp sliding regions respectively.

In [ ]:

for (i in bp_range) {
  # get the dataframe with all the indels in a region with a rAF_hi indel
  temp_var1 = paste0("filtered_df_1_", i)
  temp_var2 = paste0("filtered_df_2_", i)
  temp_var3 = paste0("filtered_df_3_", i)

  # assign a variable for the regions
  regions_df1 = paste0 ("regions_", i, "df1")
  regions_df2 = paste0 ("regions_", i, "df2")
  regions_df3 = paste0 ("regions_", i, "df3")
  
  assign (regions_df1, nrow(unique(get(temp_var1)[paste0("GID_bp", i)])))
  assign (regions_df2, nrow(unique(get(temp_var2)[paste0("GID_bp", i)])))
  assign (regions_df3, nrow(unique(get(temp_var3)[paste0("GID_bp", i)])))
  
  print(paste0("Number of regions with rAF_hi indels in ", db_name_1, " is: ", get(regions_df1)))
  print(paste0("Number of regions with rAF_hi indels in ", db_name_2, " is: ", get(regions_df2)))
  print(paste0("Number of regions with rAF_hi indels in ", db_name_3, " is: ", get(regions_df3)))
}

## Make table for above data (Supplementary Tables S3, S4, and S5). 

In [ ]:

df_1_regions_rAF_hi_indels = c(regions_10df1, regions_20df1, regions_30df1, regions_40df1)
df_1_mean_region_len = c(mean_region_len_df1_bp10, mean_region_len_df1_bp20, mean_region_len_df1_bp30, mean_region_len_df1_bp40)
df_1_median_region_len = c(median_region_len_df1_bp10, median_region_len_df1_bp20, median_region_len_df1_bp30, median_region_len_df1_bp40)
df_1_regions_prct_under_region_size = c(percent_lt_10bp_region_10df1, percent_lt_20bp_region_20df1, percent_lt_30bp_region_30df1, percent_lt_40bp_region_40df1)

df_1_summary = data.frame(bp_range, df_1_regions_rAF_hi_indels, df_1_mean_region_len, df_1_median_region_len, df_1_regions_prct_under_region_size)
df_1_summary

df_2_regions_rAF_hi_indels = c(regions_10df2, regions_20df2, regions_30df2, regions_40df2)
df_2_mean_region_len = c(mean_region_len_df2_bp10, mean_region_len_df2_bp20, mean_region_len_df2_bp30, mean_region_len_df2_bp40)
df_2_median_region_len = c(median_region_len_df2_bp10, median_region_len_df2_bp20, median_region_len_df2_bp30, median_region_len_df2_bp40)
df_2_regions_prct_under_region_size = c(percent_lt_10bp_region_10df2, percent_lt_20bp_region_20df2, percent_lt_30bp_region_30df2, percent_lt_40bp_region_40df2)

df_2_summary = data.frame(bp_range, df_2_regions_rAF_hi_indels, df_2_mean_region_len, df_2_median_region_len, df_2_regions_prct_under_region_size)
df_2_summary


df_3_regions_rAF_hi_indels = c(regions_10df3, regions_20df3, regions_30df3, regions_40df3)
df_3_mean_region_len = c(mean_region_len_df3_bp10, mean_region_len_df3_bp20, mean_region_len_df3_bp30, mean_region_len_df3_bp40)
df_3_median_region_len = c(median_region_len_df3_bp10, median_region_len_df3_bp20, median_region_len_df3_bp30, median_region_len_df3_bp40)
df_3_regions_prct_under_region_size = c(percent_lt_10bp_region_10df3, percent_lt_20bp_region_20df3, percent_lt_30bp_region_30df3, percent_lt_40bp_region_40df3)

df_3_summary = data.frame(bp_range, df_3_regions_rAF_hi_indels, df_3_mean_region_len, df_3_median_region_len, df_3_regions_prct_under_region_size)
df_3_summary 



fwrite(df_1_summary, "supplementary_table_S3.csv")
fwrite(df_2_summary, "supplementary_table_S4.csv")
fwrite(df_3_summary, "supplementary_table_S5.csv")